# 模倣学習

**参考**

* [Reinforcement Learning: Theory and Algorithms](https://rltheorybook.github.io/)
* [FLAMBE: Structural Complexity and Representation Learning of Low Rank MDPs](https://arxiv.org/abs/2006.10814)
* [Introduction to Nonparametric Estimation](https://link.springer.com/book/10.1007/b13794)

今回は模倣学習の理論を見ていきます。

**表記**

MDPを次で定義します。

1. 有限状態集合: $S=\{1, \dots, |S|\}$
2. 有限行動集合: $A=\{1, \dots, |A|\}$
3. 遷移確率行列: $P\in \mathbb{R}^{SA\times S}$
4. 報酬行列: $r\in \mathbb{R}^{S\times A}$
5. 割引率: $\gamma \in [0, 1)$
6. 初期状態: $\mu \in \mathbb{R}^{S}$

その他

* $\pi^{\star}: \mathcal{S} \mapsto \Delta(\mathcal{A})$：報酬関数で学習されたエキスパート方策
* $\widehat{\pi}: \mathcal{S} \mapsto \Delta(\mathcal{A})$：学習させる方策
* $V^{\widehat{\pi}}$を$V^\star$に近づけるのが目的
* $d^\pi$：方策$\pi$のもとでの状態と行動の訪問分布
* $D^\star=\left\{s_i^{\star}, a_i^{\star}\right\}_{i=1}^M$：$s_i^{\star}, a_i^{\star} \sim d^{\pi^{\star}}$
* $\Pi=\{\pi: \mathcal{S} \mapsto \Delta(A)\}$：方策の集合

## Behavior Cloning

基本のBehavior Cloning (BC) のサンプル効率を導出していきます。まずは$\pi^\star \in \Pi$を仮定して考えていきます。

BCでは$D^\star$以外の情報を使いません。このとき、BCでは次の最尤推定（Maximum Likelihood Estimation）で方策を学習します。

$$
\text { Behavior Cloning (BC): } \quad \widehat{\pi}=\operatorname{argmax}_{\pi \in \Pi} \sum_{i=1}^M \log \pi\left(a_i^{\star} \mid s_i^{\star}\right) \text {. }
$$

---

**最尤推定のサンプル効率(TODO)**

確率$1-\delta$以上で次の式が成立する。

$$
\mathbb{E}_{s \sim d^{\pi^{\star}}}\left\|\widehat{\pi}(\cdot \mid s)-\pi^{\star}(\cdot \mid s)\right\|_{T V}^2 \leq \frac{2 \log (|\Pi| / \delta)}{M}
$$

---
コメント

証明は[FLAMBE: Structural Complexity and Representation Learning of Low Rank MDPs](https://arxiv.org/abs/2006.10814)のTheorem 21が参考になるみたいですが、最後のところが証明が間違ってる気がしますね... 
[Empirical Processes in M-Estimation](https://www.amazon.co.jp/-/en/Sara-van-Geer/dp/0521123259)の７章に似た話が載ってるらしいので、それを見たほうがいいかもしれません。
ただし[簡易版](https://stat.ethz.ch/~geer/cowlas.pdf)の７章にあるように、求める上界自体はおおむね正しそうです。

---

証明していきましょう。

**ステップ１**

少し表記を追加します：
* $D$と独立なデータセット：${D}^\prime=\left\{s_i^{\star}, a_i^{\star}\right\}_{i=1}^M$：$s_i^{\star}, a_i^{\star} \sim d^{\pi^{\star}}$。（元論文では条件付き独立なやつを使っています）
* 誤差関数：$l(\pi, (x, y))$
* 誤差関数の和：$L(\pi, D)=\sum_{i=1}^M \ell\left(\pi,\left(x_i, y_i\right)\right)$

このとき、

$$
\mathbb{E}_{D\sim d^{\pi^\star}}\left[\exp \left(L(\hat{\pi}_D, D)-\log \mathbb{E}_{D^{\prime}\sim d^{\pi^\star}} \exp \left(L\left(\hat{\pi}_D, D^{\prime}\right)\right)-\log |\mathcal{\Pi}|\right)\right]\leq1.
$$

を証明します。

$\Pi$上の一様分布を$f$とします。また、$g:\Pi \to \mathbb{R}$を任意の関数として、$\eta(\pi):=\frac{\exp(g(\pi))}{\sum_\pi \exp(g(\pi))}$とします。$g$によって割り当てられた$\Pi$上の方策の分布です。

このとき、任意の$\Pi$上の分布$\hat{f}$について、

$$
\begin{aligned}
0 & \leq \mathrm{KL}(\hat{f} \| \eta)= \sum_\pi \hat{f}(\pi) \log (\hat{f}(\pi))-\sum_\pi \hat{f}(\pi) \log (\eta(\pi)) \\
& =\sum_\pi \hat{f}(\pi) \log (\hat{f}(\pi))+\sum_\pi \hat{f}(\pi) \log \left(\sum_{\pi^{\prime}} \exp \left(g\left(\pi^{\prime}\right)\right)\right)-\sum_\pi \hat{f}(\pi) g(\pi) \\
& =\mathrm{KL}(\hat{f} \| f)-\sum_\pi \hat{f}(\pi) g(\pi)+\log \mathbb{E}_{\pi \sim f} \exp (g(\pi)) \\
& \leq \log |\mathcal{\Pi}|-\sum_\pi \hat{f}(\pi) g(\pi)+\log \mathbb{E}_{\pi \sim f} \exp (g(\pi)) .
\end{aligned}
$$

ここで、$f$は一様分布なので$f(\pi) = \frac{1}{|\Pi|}$であり、$\mathrm{KL}(\hat{f} \| f)= \sum_\pi \hat{f}(\pi) \log (\hat{f}(\pi))+\log |\Pi| \leq \log |\Pi|$を使いました。

並び替えると、
$$
\sum_\pi \hat{f}(\pi) g(\pi)-\log |\Pi| \leq \log \mathbb{E}_{\pi \sim f} \exp (g(\pi))
$$

が成り立ちます。

<!-- ここで、$\hat{\pi}_D$を$D$を使って模倣した方策とします。 -->
また、$\hat{f}=\mathbf{1}\{\hat{\pi}_D\}$、$g(\pi)=L(\pi, D)-\log \mathbb{E}_{D^\prime\sim d^{\pi^\star}} \exp (L\left(\pi, D^\prime\right))$とします。
このとき、任意の$D$について、

$$
L(\hat{\pi}_D, D)-\log \mathbb{E}_{D^\prime \sim d^{\pi^\star}} \exp \left(L\left(\hat{\pi}_D, D^\prime\right)\right)-\log |\mathcal{\Pi}| \leq \log \mathbb{E}_{\pi \sim f} \frac{\exp (L(\pi, D))}{\mathbb{E}_{D^\prime \sim d^{\pi^\star}} \exp \left(L\left(\pi, D^{\prime}\right)\right)}
$$

が成立し、変形すると

$$
\mathbb{E}_{D\sim d^{\pi^\star}}\left[\exp \left(L(\hat{\pi}_D, D)-\log \mathbb{E}_{D^{\prime}\sim d^{\pi^\star}} \exp \left(L\left(\hat{\pi}_D, D^{\prime}\right)\right)-\log |\mathcal{\Pi}|\right)\right]
\leq \mathbb{E}_{\pi \sim f} \mathbb{E}_{D\sim d^{\pi^\star}} \frac{\exp (L(\pi, D))}{\mathbb{E}_{D^{\prime}\sim d^{\pi^\star}}\left[\exp \left(L\left(\pi, D^{\prime}\right)\right) \mid D\right]}=1 .
$$

最後の等式は$D$と$D^\prime$が独立なため成立します。
よって、

$$
\mathbb{E}_{D\sim d^{\pi^\star}}\left[\exp \left(L(\hat{\pi}_D, D)-\log \mathbb{E}_{D^{\prime}\sim d^{\pi^\star}} \exp \left(L\left(\hat{\pi}_D, D^{\prime}\right)\right)-\log |\mathcal{\Pi}|\right)\right]\leq1.
$$

**ステップ２**

$$
\mathbb{E}_{s \sim d^{\pi^\star}}\left\|\pi_1(s, \cdot)-\pi_2(s, \cdot)\right\|_{\mathrm{TV}}^2 \leq-2 \log \mathbb{E}_{s \sim d^{\pi^\star}, y \sim \pi_2(\cdot \mid s)} \exp \left(-\frac{1}{2} \log \left(\pi_2(s, y) / \pi_1(s, y)\right)\right)
$$

を証明します。まず、全変動距離を次のHellinger距離で抑えます。

$$
\mathrm{H}^2(q \| p):=\int_{\mathcal{Z}}(\sqrt{p(z)}-\sqrt{q(z)})^2 d z .
$$

[Introduction to Nonparametric Estimation](https://link.springer.com/book/10.1007/b13794)などを使えば、

$$
\|p(\cdot)-q(\cdot)\|_{\mathrm{TV}}^2 \leq \mathrm{H}^2(q \| p) \cdot\left(1-\frac{\mathrm{H}^2(q \| p)}{4}\right) \leq \mathrm{H}^2(q \| p)
$$

が成立します。また、

$$
\begin{aligned}
\mathrm{H}^2(q \| p) & =\int p(z)+q(z)-2 \sqrt{p(z) q(z)} d z=2 \cdot \mathbb{E}_{z \sim q}[1-\sqrt{p(z) / q(z)}] \\
& \leq-2 \log \mathbb{E}_{z \sim q} \sqrt{p(z) / q(z)}=-2 \log \mathbb{E}_{z \sim q} \exp \left(-\frac{1}{2} \log (q(z) / p(z))\right) .
\end{aligned}
$$

が$(1-x)\leq -\log(x)$を使うと成立します。
これに当てはめれば、
$$
\mathbb{E}_{s \sim d^{\pi^\star}}\left\|\pi_1(s, \cdot)-\pi_2(s, \cdot)\right\|_{\mathrm{TV}}^2 \leq-2 \mathbb{E}_{s \sim d^{\pi^\star}}\log \mathbb{E}_{y \sim \pi_2(\cdot \mid s)} \exp \left(-\frac{1}{2} \log \left(\pi_2(s, y) / \pi_1(s, y)\right)\right)
$$


が成立します。

**ステップ３**

確率$1-\delta$以上で次の式が成立するのを証明します。

$$
\mathbb{E}_{s \sim d^{\pi^{\star}}}\left\|\widehat{\pi}(\cdot \mid s)-\pi^{\star}(\cdot \mid s)\right\|_{T V}^2 \leq \frac{2 \log (|\Pi| / \delta)}{M}
$$

まず、Cramer-Chernoff methodを使って集中不等式を出します。
固定された$\hat{\pi}_D$について、

$$
P(L\left(\hat{\pi}_D, D\right) \geq \varepsilon) = 
P(\exp(L\left(\hat{\pi}_D, D\right)) \geq \exp(\varepsilon)) 
\leq \mathbb{E}_{D^\prime \sim d^{\pi^\star}}[\exp(L\left(\hat{\pi}_D, D^\prime\right))]\exp(-\varepsilon)
$$

が成り立ちます（最後のはマルコフの不等式です）。
$\mathbb{E}_{D^\prime \sim d^{\pi^\star}}[\exp(L\left(\hat{\pi}_D, D^\prime\right))]\exp(-\varepsilon) = \delta$を解けば、
確率$\delta$以下で

$$L\left(\hat{\pi}_D, D\right) \geq \log \mathbb{E}_{D^\prime \sim d^{\pi^\star}}[\exp(L\left(\hat{\pi}_D, D^\prime\right))] - \log \delta $$

$\hat{\pi}_D$を$\Pi$についてUnion Boundを取れば、任意の$\hat{\pi}_D$について、確率$1-\delta$以上で

$$-\log \mathbb{E}_{D^\prime \sim d^{\pi^\star}}[\exp(L\left(\hat{\pi}_D, D^\prime\right))] \leq -L\left(\hat{\pi}_D, D\right) + \log |\Pi| + \log (1/\delta) $$

最後に
$
L(\pi, D) = -\frac{1}{2}\sum_{i=1}^M \log \frac{\pi^\star\left(a_i^{\star} \mid s_i^{\star}\right)}{\pi\left(a_i^{\star} \mid s_i^{\star}\right)}
$
とします。このとき、上の不等式の右辺は

$$
\begin{aligned}
-L\left(\hat{\pi}_D, D\right) + \log |\Pi| + \log (1/\delta)
&=-\frac{1}{2}\sum_{i=1}^M \log \frac{\pi^\star\left(a_i^{\star} \mid s_i^{\star}\right)}{\hat{\pi}_D\left(a_i^{\star} \mid s_i^{\star}\right)} + \log |\Pi| + \log (1/\delta)\\
&=-\frac{1}{2}\sum_{i=1}^M \log \pi^\star\left(a_i^{\star} \mid s_i^{\star}\right) + \frac{1}{2}\sum_{i=1}^M \log {\hat{\pi}_D\left(a_i^{\star} \mid s_i^{\star}\right)} + \log |\Pi| + \log (1/\delta)\\
&\leq \log |\Pi| + \log (1/\delta)
\end{aligned}
$$

が成立します。最後の不等式は$\hat{\pi}_D$が$D$上の最尤推定なせいです。
一方で、左辺は

$$
\begin{aligned}
-\log \mathbb{E}_{D^\prime \sim d^{\pi^\star}}[\exp(L\left(\hat{\pi}_D, D^\prime\right))] &= 
-\log \mathbb{E}_{D^\prime \sim d^{\pi^\star}}\left[\exp\left(-\frac{1}{2}\sum_{i=1}^M \log \frac{\pi^\star\left(a_i^{\star} \mid s_i^{\star}\right)}{\hat{\pi}_D\left(a_i^{\star} \mid s_i^{\star}\right)}\right)\right] \\
&\geq M\frac{1}{2}\mathbb{E}_{s \sim d^{\pi^\star}}\left\|\pi_1(s, \cdot)-\pi_2(s, \cdot)\right\|_{\mathrm{TV}}^2 
\end{aligned}
$$

TODO: ここの変形ができなかったのでギブアップです... 正しいやり方を見つけたら追記します。

---

最尤推定のサンプル効率を使えば、BCのサンプル効率が出せます。

**BCのサンプル効率**

確率$1-\delta$以上で、BCは次を満たす方策$\hat{\pi}$を返します：

$$V^{\star}-V^{\widehat{\pi}} \leq \frac{3}{(1-\gamma)^2} \sqrt{\frac{\ln (|\Pi| / \delta)}{M}}$$

証明は簡単です：

$$
\begin{aligned}
& (1-\gamma)\left(V^{\star}-V^{\widehat{\pi}}\right)=\mathbb{E}_{s \sim d^{\pi^{\star}}} \mathbb{E}_{a \sim \pi^{\star}(\cdot \mid s)} A^{\widehat{\pi}}(s, a) \\
& =\mathbb{E}_{s \sim d^{\pi^{\star}}} \mathbb{E}_{a \sim \pi^{\star}(\cdot \mid s)} A^{\widehat{\pi}}(s, a)-\mathbb{E}_{s \sim d^{\pi^{\star}}} \mathbb{E}_{a \sim \widehat{\pi}(\cdot \mid s)} A^{\widehat{\pi}}(s, a) \\
& \leq \mathbb{E}_{s \sim d^{\pi^{\star}}} \frac{1}{1-\gamma}\left\|\pi^{\star}(\cdot \mid s)-\widehat{\pi}(\cdot \mid s)\right\|_1 \\
& \leq \frac{1}{1-\gamma} \sqrt{\mathbb{E}_{s \sim d^{\pi^{\star}}}\left\|\pi^{\star}(\cdot \mid s)-\widehat{\pi}(\cdot \mid s)\right\|_1^2} \\
& =\frac{1}{1-\gamma} \sqrt{4 \mathbb{E}_{s \sim d^{\pi^{\star}}}\left\|\pi^{\star}(\cdot \mid s)-\widehat{\pi}(\cdot \mid s)\right\|_{t v}^2} . \\
&
\end{aligned}
$$

ここで、$\sup _{s, a, \pi}\left|A^\pi(s, a)\right| \leq \frac{1}{1-\gamma}$と$(\mathbb{E}[x])^2 \leq \mathbb{E}\left[x^2\right]$を使っています。
最後に最尤推定のバウンドを適用すれば終わりです。

##  Distribution Matching with Scheffe Tournament (DM-ST)

遷移確率なしでは性能のバウンドが$\frac{1}{(1-\gamma)^2}$に比例していました。実はこの二乗への依存はBCでは回避できず、Distribution shift問題と呼ばれています。

一方で、MDPの遷移確率$P$を使っていい場合はこれは回避できます。
まずは計算コストが重いアルゴリズムについて見てみましょう。

**表記**

* Witness関数： $f_{\pi, \pi^{\prime}}:=\operatorname{argmax}_{f:\|f\|_{\infty} \leq 1}\left[\mathbb{E}_{s, a \sim d^\pi} f(s, a)-\mathbb{E}_{s, a \sim d^{\pi^{\prime}}} f(s, a)\right]$
    * $\pi$で動いたデータ$\mathbb{E}_{s, a \sim d^\pi} f(s, a)$をなるべく最大化して、$\pi^\prime$で動いたデータ$\mathbb{E}_{s, a \sim d^{\pi^{\prime}}} f(s, a)$をなるべく最小化する関数です。
* Witness関数の集合：$\mathcal{F}=\left\{f_{\pi, \pi^{\prime}}: \pi, \pi^{\prime} \in \Pi, \pi \neq \pi^{\prime}\right\}$
    * $|\mathcal{F}| \leq |\Pi|^2$が成立してます。

DM-STは次の式で方策を計算します。

$$
\text { DM-ST: } \widehat{\pi} \in \operatorname{argmin}_{\pi \in \Pi}\left[\max _{f \in \mathcal{F}}\left[\mathbb{E}_{s, a \sim d^\pi} f(s, a)-\frac{1}{M} \sum_{i=1}^M f\left(s_i^{\star}, a_i^{\star}\right)\right]\right]
$$

---

直感的な説明をしてみます。
まず、$\mathcal{F}$は$\arg \max _{f:\|f\|_{\infty} \leq 1}\left[\mathbb{E}_{s, a \sim d^\pi} f(s, a)-\mathbb{E}_{s, a \sim d^{\star}} f(s, a)\right]$を含んでいるので、
$$
\max _{f \in \mathcal{F}}\left[\mathbb{E}_{s, a \sim d^\pi} f(s, a)-\mathbb{E}_{s, a \sim d^{\star}} f(s, a)\right]=\max _{f:\|f\|_{\infty} \leq 1}\left[\mathbb{E}_{s, a \sim d^\pi} f(s, a)-\mathbb{E}_{s, a \sim d^{\star}} f(s, a)\right]=\left\|d^\pi-d^{\pi^{\star}}\right\|_1
$$
が成り立っています。$M$が大きいとき、DM-STは$d^\pi$と$d^{\pi^\star}$との全変動距離を一番小さくする方策を学習しようとします。

---

DM-STは確率$1-\delta$で次の式を満たす方策を返します：
$$V^{\star}-V^{\widehat{\pi}} \leq \frac{4}{1-\gamma} \sqrt{\frac{2 \ln (|\Pi|)+\ln \left(\frac{1}{\delta}\right)}{M}}$$

証明していきましょう。
Hoeffdingの不等式とUnion Boundから、確率$1-\delta$以上で

$$
\left|\frac{1}{M} \sum_{i=1}^M f\left(s_i^{\star}, a_i^{\star}\right)-\mathbb{E}_{s, a \sim d^{\star}} f(s, a)\right| \leq 2 \sqrt{\frac{\ln (|\mathcal{F}| / \delta)}{M}}:=\epsilon_{\text {stat }}
$$

ここで、

* $\widehat{f}:=\arg \max _{f \in \mathcal{F}}\left[\mathbb{E}_{s, a \sim d^{\widehat{\pi}}} f(s, a)-\mathbb{E}_{s, a \sim d^{\star}} f(s, a)\right]$
* $\widetilde{f}:=\arg \max _{f \in \mathcal{F}} \mathbb{E}_{s, a \sim d^{\hat{\pi}}} f(s, a)-\frac{1}{M} \sum_{i=1}^M f\left(s_i, a_i\right)$

とします。

$$
\begin{aligned}
\left\|d^{\widehat{\pi}}-d^{\star}\right\|_1 & =\mathbb{E}_{s, a \sim d^{\widehat{\pi}}} \widehat{f}(s, a)-\mathbb{E}_{s, a \sim d^{\star}} \widehat{f}(s, a) \leq \mathbb{E}_{s, a \sim d^{\widehat{\pi}}} \widehat{f}(s, a)-\frac{1}{M} \sum_{i=1}^M \widehat{f}\left(s_i^{\star}, a_i^{\star}\right)+\epsilon_{\text {stat }} \\
& \leq \mathbb{E}_{s, a \sim d^{\widehat{\pi}}} \widetilde{f}(s, a)-\frac{1}{M} \sum_{i=1}^M \widetilde{f}\left(s_i, a_i\right)+\epsilon_{s t a t} \\
& \leq \mathbb{E}_{s, a \sim d^{\pi^{\star}}} \widetilde{f}(s, a)-\frac{1}{M} \sum_{i=1}^M \widetilde{f}\left(s_i, a_i\right)+\epsilon_{\text {stat }} \\
& \leq \mathbb{E}_{s, a \sim d^{\pi^{\star}}} \widetilde{f}(s, a)-\mathbb{E}_{s, a \sim d^{\star}} \widetilde{f}(s, a)+2 \epsilon_{\text {stat }}=2 \epsilon_{\text {stat }},
\end{aligned}
$$

以上より、　

$$
V^{\widehat{\pi}}-V^{\star}=\frac{1}{1-\gamma}\left(\mathbb{E}_{s, a \sim d^{\widehat{\pi}}} r(s, a)-\mathbb{E}_{s, a \sim d^{\star}} r(s, a)\right) \leq \frac{\sup _{s, a}|r(s, a)|}{1-\gamma}\left\|d^{\widehat{\pi}}-d^{\star}\right\|_1 \leq \frac{2}{1-\gamma} \epsilon_{s t a t}
$$

を得ます。